<a href="https://colab.research.google.com/github/HEsiyun/cv-lab3/blob/main/cv_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.1/877.1 kB 22.5 MB/s eta 0:00:00


In [2]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
!rm -rf /content/dataset
!rm -rf /content/yolov8_dataset
!rm -rf /content/darknet
!rm -rf /content/yolov8_finetune

In [4]:
!nvidia-smi

Thu Oct 24 03:56:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
import gdown
import zipfile

# Google Drive file ID
file_id = "19W45cMfvBnhV0vqBVJgUHHpe7WC1YLdG"
url = f"https://drive.google.com/uc?export=download&id={file_id}"

# Download the file using gdown
gdown.download(url, "/content/dataset.zip", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?export=download&id=19W45cMfvBnhV0vqBVJgUHHpe7WC1YLdG
From (redirected): https://drive.google.com/uc?export=download&id=19W45cMfvBnhV0vqBVJgUHHpe7WC1YLdG&confirm=t&uuid=83fe5ddb-8c01-4abc-8f6e-e840542cf6ad
To: /content/dataset.zip
100%|██████████| 211M/211M [00:02<00:00, 93.0MB/s]


'/content/dataset.zip'

In [6]:
# Extract the zip file
try:
    with zipfile.ZipFile("/content/dataset.zip", 'r') as zip_ref:
        zip_ref.extractall("/content/dataset")
    print("Dataset downloaded and unzipped successfully.")
except zipfile.BadZipFile:
    print("Error: The downloaded file is not a valid zip file.")

Dataset downloaded and unzipped successfully.


In [7]:
import os
import shutil
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split

# Paths to your dataset
images_dir = '/content/dataset/filtered_images'
annotations_dir = '/content/dataset/filtered_annotations'
# Output directory for YOLOv8 training data
output_dir = '/content/yolov8_dataset'
os.makedirs(output_dir, exist_ok=True)

# Create directories for train, val, and test
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')
test_dir = os.path.join(output_dir, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Get a list of image files
image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg') or f.endswith('.png')]

# Split the data into train, validation, and test sets
train_files, temp_files = train_test_split(image_files, test_size=0.3, random_state=42)  # 70% train
val_files, test_files = train_test_split(temp_files, test_size=0.5, random_state=42)  # 15% val, 15% test

# Function to convert XML annotations to YOLO format and save as .txt
def convert_xml_to_yolo(xml_file, img_width, img_height):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    yolo_lines = []
    for obj in root.findall('object'):
        class_name = obj.find('name').text
        # Convert class name to an index (You may want to create a mapping)
        class_id = 0

        bndbox = obj.find('bndbox')
        xmin = float(bndbox.find('xmin').text)
        ymin = float(bndbox.find('ymin').text)
        xmax = float(bndbox.find('xmax').text)
        ymax = float(bndbox.find('ymax').text)

        # Convert to YOLO format (normalized coordinates)
        x_center = (xmin + xmax) / 2 / img_width
        y_center = (ymin + ymax) / 2 / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height

        yolo_lines.append(f"{class_id} {x_center} {y_center} {width} {height}")

    return yolo_lines

# Function to copy images and corresponding annotations
def copy_files_and_convert_labels(file_list, source_dir, dest_dir, annotation_dir):
    for filename in file_list:
        # Copy image files
        src_img_path = os.path.join(source_dir, filename)
        dest_img_path = os.path.join(dest_dir, filename)
        shutil.copy(src_img_path, dest_img_path)

        # Get image dimensions
        img_width, img_height = 600, 600

        # Convert and save corresponding annotation files
        base_name = os.path.splitext(filename)[0]
        src_label_path = os.path.join(annotation_dir, base_name + '.xml')
        if os.path.exists(src_label_path):
            yolo_lines = convert_xml_to_yolo(src_label_path, img_width, img_height)
            dest_label_path = os.path.join(dest_dir, base_name + '.txt')
            with open(dest_label_path, 'w') as f:
                f.write('\n'.join(yolo_lines))

# Copy files to their respective directories and convert labels
copy_files_and_convert_labels(train_files, images_dir, train_dir, annotations_dir)
copy_files_and_convert_labels(val_files, images_dir, val_dir, annotations_dir)
copy_files_and_convert_labels(test_files, images_dir, test_dir, annotations_dir)

print("Dataset organized and annotations converted to YOLO format:")
print(f"- Train images: {len(train_files)}")
print(f"- Validation images: {len(val_files)}")
print(f"- Test images: {len(test_files)}")

Dataset organized and annotations converted to YOLO format:
- Train images: 2993
- Validation images: 642
- Test images: 642


In [8]:
import os

# Define the base directory where your dataset is located
base_dir = "/content/yolov8_dataset"

# Define the paths for train, val, and test directories
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")
test_dir = os.path.join(base_dir, "test")

# Define the class names
import os
import xml.etree.ElementTree as ET

# Define the path to the annotations directory
annotations_dir = '/content/dataset/filtered_annotations'

# Convert the set to a sorted list
class_names = [0]

# Create the data.yaml content
data_yaml_content = f"""
# YOLOv8 data configuration file

# Base directory of the dataset
path: {base_dir}

# Paths for training, validation, and testing image directories
train: {train_dir}
val: {val_dir}
test: {test_dir}

# Number of classes
nc: {len(class_names)}

# Class names
names:
"""

# Add class names to the YAML content
for i, name in enumerate(class_names):
    data_yaml_content += f"  {i}: '{name}'\n"

# Write the content to data.yaml
data_yaml_path = os.path.join('/content/yolov8_dataset', "data.yaml")
with open(data_yaml_path, "w") as file:
    file.write(data_yaml_content)

print("data.yaml file has been created successfully at:", data_yaml_path)

data.yaml file has been created successfully at: /content/yolov8_dataset/data.yaml


In [12]:
# prompt: fine-tune last layer of the train dataset for 3 epochs with 1 warmup, output performance on val in each epoch

from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Define the path to the data.yaml file
data_yaml_path = '/content/yolov8_dataset/data.yaml'

# Fine-tune the model on the custom dataset
results = model.train(
    data=data_yaml_path,          # Path to your dataset configuration (data.yaml)
    epochs=30,                    # Increased number of epochs for fine-tuning
    imgsz=640,                    # Image size for training
    patience=10,                  # Early stopping patience (if no improvement after 10 epochs)
    device=[0],                   # Specify the device (e.g., GPU 0)
    optimizer='Adam',             # Using Adam optimizer for training
    lr0=0.0001,                   # Lower initial learning rate for fine-tuning
    lrf=0.01,                     # Final learning rate (after warm-up)
    warmup_epochs=2,              # Warm-up for 2 epochs (to avoid abrupt changes in gradients)
    freeze=[0, 1, 2, 3],          # Freezing all layers except the last layer (adjust if needed)
)

Ultralytics 8.3.21 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/yolov8_dataset/data.yaml, epochs=30, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=[0, 1, 2, 3], multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tru

wandb: Currently logged in as: hesiyun0603 (hesiyun0603-no). Use `wandb login --relogin` to force relogin


Freezing layer 'model.0.conv.weight'
Freezing layer 'model.0.bn.weight'
Freezing layer 'model.0.bn.bias'
Freezing layer 'model.1.conv.weight'
Freezing layer 'model.1.bn.weight'
Freezing layer 'model.1.bn.bias'
Freezing layer 'model.2.cv1.conv.weight'
Freezing layer 'model.2.cv1.bn.weight'
Freezing layer 'model.2.cv1.bn.bias'
Freezing layer 'model.2.cv2.conv.weight'
Freezing layer 'model.2.cv2.bn.weight'
Freezing layer 'model.2.cv2.bn.bias'
Freezing layer 'model.2.m.0.cv1.conv.weight'
Freezing layer 'model.2.m.0.cv1.bn.weight'
Freezing layer 'model.2.m.0.cv1.bn.bias'
Freezing layer 'model.2.m.0.cv2.conv.weight'
Freezing layer 'model.2.m.0.cv2.bn.weight'
Freezing layer 'model.2.m.0.cv2.bn.bias'
Freezing layer 'model.3.conv.weight'
Freezing layer 'model.3.bn.weight'
Freezing layer 'model.3.bn.bias'
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 102MB/s]


AMP: checks passed ✅


...

                   all        642       2065      0.888      0.827      0.908      0.816



30 epochs completed in 0.574 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.21 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:11<00:00,  1.88it/s]


                   all        642       2065      0.885      0.819      0.906      0.816
Speed: 0.3ms preprocess, 2.4ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to runs/detect/train


lr/pg0,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▄███▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
lr/pg2,▄███▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
metrics/mAP50(B),▁▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇███████
metrics/mAP50-95(B),▁▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇███████
metrics/precision(B),▄▆▆▇▇▇▇▇▇▇▇▇▇▆▇▇▆▇▇▅▅██▂▁▃█▁▁▁
metrics/recall(B),▁▂▂▃▃▃▃▃▃▃▃▃▃▄▃▃▄▃▄▅▅▃▃██▆▃██▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▆▅▅▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▃▂▂▂▁▁▁▁▁▁▂


In [13]:
from ultralytics import YOLO

# Load the best model from the training run
best_model = YOLO('/content/yolov8_finetune/train/weights/best.pt')

# Run inference on the test dataset
results = best_model.val(data='/content/yolov8_dataset/data.yaml')

# Print essential results
print("Validation Results:")
print(f" - mAP: {results.box.map.mean():.4f}")  # Mean Average Precision
print(f" - AP50: {results.box.ap50.mean():.4f}")  # Average Precision at IoU=0.50
print(f" - Precision: {results.box.p.mean():.4f}")  # Precision
print(f" - Recall: {results.box.r.mean():.4f}")  # Recall

FileNotFoundError: [Errno 2] No such file or directory: '/content/yolov8_finetune/train/weights/best.pt'